In [57]:
#profiler load 
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../..')
from pyprofiler.utils import config_utils
import pyprofiler.utils.goatools_utils as goa
import pyprofiler.profiler as profiler
from qtlsearch.OBOParser import OBO
import pandas as pd
import numpy as np
#select rand profile w annotations
#mask annotations
#pull in coevolving Hogs
#feed profile vectors and GO to NN
#Try to recover annotations

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
#lets also create a go annotation enrichment study object to describe functional enrichment in our profiling results
gaf = goa.buildGAF( '/home/cactuskid13/mntpt/unil_backup/profilingbackup/gaf/oma-go.txt' )

In [59]:
p = profiler.Profiler(lshforestpath = config_utils.datadir + '/latest_all_256/newlshforest.pkl'  , hashes_h5=config_utils.datadir+ 'latest_all_256/hashes.h5'  , mat_path= None, oma = '/home/cactuskid13/mntpt/OMA/latest/OmaServer.h5', tar= None , mastertree =config_utils.datadir + 'latest_all_256_master_tree.pkl' , nsamples = 256)

loading lsh
indexing lsh


Cannot load SequenceSearch. Any future call to seq_search will fail!
Traceback (most recent call last):
  File "../../../pyprofiler/pyoma/browser/db.py", line 879, in __init__
    self.seq_idx = self.seq_idx()
  File "/home/cactuskid13/miniconda3/envs/pyprofiler3/lib/python3.7/site-packages/tables/link.py", line 397, in __call__
    self.extfile = tables.open_file(filename, **kwargs)
  File "/home/cactuskid13/miniconda3/envs/pyprofiler3/lib/python3.7/site-packages/tables/file.py", line 320, in open_file
    return File(filename, mode, title, root_uep, filters, **kwargs)
  File "/home/cactuskid13/miniconda3/envs/pyprofiler3/lib/python3.7/site-packages/tables/file.py", line 784, in __init__
    self._g_new(filename, mode, **params)
  File "tables/hdf5extension.pyx", line 374, in tables.hdf5extension.File._g_new
  File "/home/cactuskid13/miniconda3/envs/pyprofiler3/lib/python3.7/site-packages/tables/utils.py", line 157, in check_file_access
    raise IOError("``%s`` does not exist" % (fil

3278
DONE


In [60]:
IDs = [  'ARATH26614'  ]

def grabHog(ID):
    try:
        return p.db_obj.entry_by_entry_nr(p.db_obj.id_resolver.resolve(ID))[4].decode()
    except:
        return np.nan
Hogs = dict( zip ( IDs, [ grabHog(i) for i in IDs ]))

print(Hogs)

{'ARATH26614': 'HOG:0479378.1b.9a'}


In [61]:
print(p.db_obj.id_resolver.resolve('ARATH26614'))

print( p.db_obj.entry_by_entry_nr(14336066))

14336066
(14336066, 5743470832, 706, 744415, b'HOG:0479378.1b.9a', b'4', 7063291, 7066784, 1, 0, b'HAP2_ARATH', 17244748561, 2119, b'64dcfddd880e7d4265a77286668f7be9', 679800248, 25, b'', -1, -1)


In [62]:
tab = p.db_obj.db.get_node('/Protein/Entries') 
print(tab.nrows)
print(tab[0:10])
hogs = { r[4].decode().split('.')[0] for r in tab }
print(len(hogs))

14674630
[( 1,    0, 632, 899681, b'HOG:0020846.1b', b'NJBF01000001',  234, 2129,  1, 0, b'A0A2U3CKJ0',    0, 1897, b'7b5ef3d429cfe701293ce1da82a52e38',    0, 115, b'', -1, -1)
 ( 2,  632, 183,      0, b'HOG:0682186.1c', b'NJBF01000001', 2326, 2874,  1, 0, b'A0A2U3CKK2', 1897,  550, b'6f9a42e07f9b5a4b21205dd148d41a2e',  115, 113, b'', -1, -1)
 ( 3,  815, 188,      0, b'HOG:0682186.1b', b'NJBF01000001', 2888, 3451,  1, 0, b'A0A2U3CKK9', 2447,  565, b'32b90cdaaaa2f1b4828d78bc3e24037d',  228, 113, b'', -1, -1)
 ( 4, 1003, 261,      0, b'', b'NJBF01000001', 3474, 4256, -1, 0, b'A0A2U3CKI6', 3012,  784, b'ac5f57196d89a79373192c56d04ceaa4',  341, 107, b'', -1, -1)
 ( 5, 1264, 191,      0, b'', b'NJBF01000001', 4350, 4922, -1, 0, b'A0A2U3CKJ5', 3796,  574, b'6bace8b6e08bfe08398bfb88baaa6e52',  448, 113, b'', -1, -1)
 ( 6, 1455, 184,      0, b'', b'NJBF01000001', 5018, 5569, -1, 0, b'A0A2U3CKL8', 4370,  553, b'2809bdc6745f6e5d4f2cdc9714081c92',  561, 113, b'', -1, -1)
 ( 7, 1639, 263,      0, 

In [63]:
import time

def yeild_annot_hogs(p, hogs, chunksize = 10 , verbose = False):
    datachunkdict = {}
    datadf = None
    for h in hogs:
        if len(h)> 0:
            if datadf is not None and len(datadf) > 0 and len(datadf.hog.unique())> chunksize:
                datadf['taxid'] = datadf.genome.map( lambda x : x.ncbi_taxon_id)
                yield datadf
                datachunkdict = {}
                datadf = None
            if verbose :
                print(h)
            datachunkdict= { m.omaid:{ 'genome': m.genome , 'annot': gaf[m.omaid] , 'hog':h } for m in p.db_obj.iter_members_of_hog_id(h) if m.omaid in gaf }
            if datadf is None:
                datadf = pd.DataFrame.from_dict( datachunkdict , orient = 'index' )
            else:
                datadf = pd.concat([datadf , pd.DataFrame.from_dict( datachunkdict , orient = 'index' ) ])
hogiterator = yeild_annot_hogs( p, hogs )
for i in range(5):
    t0 = time.time()
    print( next(hogiterator))
    print( time.time()-t0)
    


                              genome  \
ANOGA11650     <Genome(ANOGA, 7165)>   
CULSO11242   <Genome(CULSO, 179676)>   
DROAN08980     <Genome(DROAN, 7217)>   
DROEL01323    <Genome(DROEL, 30023)>   
DROER07857     <Genome(DROER, 7220)>   
DROKI02614    <Genome(DROKI, 30033)>   
DROME06277     <Genome(DROME, 7227)>   
DROSE12265     <Genome(DROSE, 7238)>   
DROSI04053     <Genome(DROSI, 7240)>   
DROTK10557    <Genome(DROTK, 29030)>   
DROER02850     <Genome(DROER, 7220)>   
DROME15656     <Genome(DROME, 7227)>   
DROPS13785    <Genome(DROPS, 46245)>   
ARCFU00979   <Genome(ARCFU, 224325)>   
ARCVS01882   <Genome(ARCVS, 693661)>   
DEHLB00747   <Genome(DEHLB, 552811)>   
DEHM101366   <Genome(DEHM1, 243164)>   
DEHMB01260   <Genome(DEHMB, 216389)>   
DEHMC01215   <Genome(DEHMC, 255470)>   
DEHMG01152   <Genome(DEHMG, 633145)>   
DEHMV01186   <Genome(DEHMV, 311424)>   
FERPA00789   <Genome(FERPA, 589924)>   
HALBP02346   <Genome(HALBP, 469382)>   
HALHT00703   <Genome(HALHT, 634497)>   


In [64]:
targets = pd.read_table( './out_map_by_hog.tsv')
targets = targets.sort_values( by= 'oma_id')
targets['species'] = targets.oma_id.map( lambda x : x[0:5])
print(targets.head())
species = pd.read_table( './oma-species.txt' , sep = '\t' , comment = '#', header = None)
specdict = dict( zip(species[0] , species[1]) )
targets['taxcode'] = targets.species.map(specdict)
print(targets)



/home/cactuskid13/miniconda3/envs/pyprofiler3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.
/home/cactuskid13/miniconda3/envs/pyprofiler3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: read_table is deprecated, use read_csv instead.
  """


            cafa_id      oma_id     fam species
57070  T37020001074  ARATH00006  472976   ARATH
69814  T37020003425  ARATH00010  483443   ARATH
70000  T37020007063  ARATH00011  483592   ARATH
3798   T37020007582  ARATH00013  182502   ARATH
3236   T37020015139  ARATH00015  177269   ARATH
              cafa_id      oma_id     fam species  taxcode
57070    T37020001074  ARATH00006  472976   ARATH     3702
69814    T37020003425  ARATH00010  483443   ARATH     3702
70000    T37020007063  ARATH00011  483592   ARATH     3702
3798     T37020007582  ARATH00013  182502   ARATH     3702
3236     T37020015139  ARATH00015  177269   ARATH     3702
82542    T37020009232  ARATH00020  684175   ARATH     3702
57872    T37020012135  ARATH00021  473862   ARATH     3702
67542    T37020007239  ARATH00025  481745   ARATH     3702
61295    T37020002614  ARATH00029  476412   ARATH     3702
4952     T37020005999  ARATH00033  187249   ARATH     3702
54778    T37020011574  ARATH00039  470677   ARATH     3702
5681

In [65]:
from matplotlib import pyplot as plt

In [65]:
res = p.hog_query(hog_id='HOG:0479378' , k = 100)
print(res)
res +=['479378']
qhog = 'HOG:0479378'

from goatools import obo_parser
obo = './go.obo'

def get_ancestors( golist , obo,  levels = 4 ):
    ancestors = []
    
    for l in range( levels):
        if l == 0 :
            a = [ obo[p.id].id for g in golist for p in obo[g].parents ]
        else:
            a = [ obo[p.id].id for g in a for p in obo[g].parents ]
        ancestors += a
    ancestors += golist
    return [goa.goterm2id(goterm) for goterm in set(ancestors)]

def get_ancestors_all( golist , obo):
    ancestors = []
    lastlen = -1
    l = 0
    while len(ancestors)>lastlen:
        if l == 0 :
            a = [ obo[p.id].id for g in golist for p in obo[g].parents ]
            
        else:
            a = [ obo[p.id].id for g in a for p in obo[g].parents ]
        lastlen = len(ancestors)
        ancestors += a
        l+=1
    
    ancestors += golist
    return [goa.goterm2id(goterm) for goterm in set(ancestors)]


obo = obo_parser.GODag(obo)
res =[ "HOG:" + (7-len(str(fam_id))) * '0' + str(fam_id) for fam_id in res ]
godict = { r:  { m.omaid:gaf[m.omaid]  for m in p.db_obj.iter_members_of_hog_id(r) if m.omaid in gaf} for r in res}
godict = { r: get_ancestors ( set([ g  for go in godict[r].values() for g in go] ) , obo ) for r in godict }
hashes =  p.pull_hashes(res)
jaccards = { r:hashes[r].jaccard(hashes[qhog]) for r in hashes }

#try multiple readers 
#note... pytables i useless no concurrent reading allowed
'''from multiprocessing import Process, Queue 

from functools import partial

inq = Queue()
outq = Queue()

def retreiveGO(dbobj, inq , outq , c):
    nextval = 1
    retdict = {}
    print('starting ' + str(c))
    while nextval is not None:
        nextval = inq.get()
        if nextval:
            retdict.update( { nextval : { m.omaid : dbobj.get_gene_ontology_annotations(m.entry_nr)  for m in p.db_obj.iter_members_of_hog_id(nextval) } } )
            if len(retdict)> 10:
                outq.put(retdict)
                retdict = {}
    outq.put(None)
    print('done process' + str(c))

cpus = 4
processes = []

for c in range(cpus):
    
    proc = Process(target=retreiveGO , args=( p.db_obj , inq , outq , c) )
    proc.start()
    processes.append(proc)

annotations = {}
for i,r in enumerate(res):
    inq.put(r)
    if outq.empty() == False:
        retdict = outq.get()
        annotations.update(retdict)

for c in range(cpus):
    inq.put(None)

for val in outq:
    if val:
         annotations.update(val)
'''

['474864', '465608', '481019', '482039', '480400', '480344', '476304', '479378', '467484', '466906', '466364', '469002', '477422', '478366', '481341', '677929', '473995', '474738', '481809', '477272', '483457', '470089', '679843', '483176', '468873', '477088', '470107', '678897', '474920', '467904', '475158', '469814', '681490', '463766', '467945', '474900', '681625', '679689', '471940', '474823', '463258', '679936', '481769', '480929', '467312', '471342', '481275', '677371', '479201', '679321', '469432', '483165', '483299', '466304', '483601', '465913', '470320', '470721', '481244', '467544', '473994', '679081', '677970', '481595', '475896', '469398', '478942', '681467', '676758', '476518', '479455', '477013', '478353', '467749', '465874', '464912', '481506', '485092', '465670', '463976', '464184', '466122', '677737', '482822', '478803', '481941', '474534', '479265', '482656', '477557', '480155', '472090', '467777', '473628', '481956', '477880', '474554', '463113', '475701', '472225']

"from multiprocessing import Process, Queue \n\nfrom functools import partial\n\ninq = Queue()\noutq = Queue()\n\ndef retreiveGO(dbobj, inq , outq , c):\n    nextval = 1\n    retdict = {}\n    print('starting ' + str(c))\n    while nextval is not None:\n        nextval = inq.get()\n        if nextval:\n            retdict.update( { nextval : { m.omaid : dbobj.get_gene_ontology_annotations(m.entry_nr)  for m in p.db_obj.iter_members_of_hog_id(nextval) } } )\n            if len(retdict)> 10:\n                outq.put(retdict)\n                retdict = {}\n    outq.put(None)\n    print('done process' + str(c))\n\ncpus = 4\nprocesses = []\n\nfor c in range(cpus):\n    \n    proc = Process(target=retreiveGO , args=( p.db_obj , inq , outq , c) )\n    proc.start()\n    processes.append(proc)\n\nannotations = {}\nfor i,r in enumerate(res):\n    inq.put(r)\n    if outq.empty() == False:\n        retdict = outq.get()\n        annotations.update(retdict)\n\nfor c in range(cpus):\n    inq.put(Non

In [66]:
plt.hist(jaccards.values(), 20)
print(sorted(jaccards.values()))
#each annotation set has a weight with jaccard...


NameError: name 'plt' is not defined

In [70]:
#load go2vec
import numpy as np
from keras.models import *
from keras.optimizers import *
from keras.layers import *
from keras.metrics import *
model = print('Load the model..')
modelfile = '../models/GO2vec12_20_2019_13_17_50_507523.h5'
model = load_model(modelfile)
inputlayer = Input((1,) , name='target_in')
layer = model.get_layer('embedding')
x = layer(inputlayer)
embedder = Model( inputs=[inputlayer] , outputs =[x]  )
embedder.build( input_shape = (1,) )
print(embedder.summary())

Load the model..
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
target_in (InputLayer)       (None, 1)                 0         
_________________________________________________________________
embedding (Embedding)        (None, 1, 5)              88875     
Total params: 88,875
Trainable params: 88,875
Non-trainable params: 0
_________________________________________________________________
None


In [68]:
import pickle
with open( '../models/gafobects.pkl' , 'rb' ) as gafstats:
    nterms , c , index , reverse_index , sampling = pickle.loads(gafstats.read())
print(sum(c.values()))
print(len(index))
min_annot= 50
index = { t:index[t] for t in index if c[t] > min_annot }
c = { t:c[t] for t in index }
nterms = len(index)
print(len(index))
embedmat = embedder.predict( [np.array(list(index.values()) ) ] )
embedmat = embedmat.reshape(nterms,-1)
print(embedmat)
print(embedmat.shape)
from sklearn.neighbors import KDTree as KD
tree = KD( embedmat )
print(tree)



11603854834
33703
17775
[[ 162.7191   -113.12968  -171.0941    173.88841   116.04149 ]
 [  74.479225  -74.467575  -71.74709    72.79818    65.72799 ]
 [  29.158852  -32.257793  -18.611912   28.79605    32.13372 ]
 ...
 [   0.          0.          0.          0.          0.      ]
 [   0.          0.          0.          0.          0.      ]
 [   0.          0.          0.          0.          0.      ]]
(17775, 5)


In [69]:
#free up the gpu
del embedder


In [71]:
#use the embedder on sets of go terms
govecs ={ r : embedder.predict( np.asarray([ index[go] for go in godict[r] if go in index ]) ) for r in godict }

In [72]:

print(obo['GO:0000002'].name)
go = obo['GO:0000002']
print( get_ancestors([go.id] , obo ) )

mitochondrial genome maintenance
[9987, 16043, 71840, 2, 6996, 7005]


In [73]:
import random

def id2GO(intID):
    return 'GO:{:07d}'.format(intID)


for go in godict['HOG:0479378']:
    print(obo[goa.id2goterm(go)].name)

print( get_ancestors( [ id2GO(g) for g in godict['HOG:0479378']], obo) )


for i in range(govecs['HOG:0479378'].shape[0]):
    dist, ind = tree.query( govecs['HOG:0479378'][i] , k=10)    
    print('returned terms')
    #print(obo[list(godict['HOG:0479378'])[i]].name)
    print([ obo[ id2GO(int(reverse_index[Id]))].name for Id in list(ind[0]) if id2GO(int(reverse_index[Id])) in obo ] )
    print('count')
    print([ c[reverse_index[Id]] for Id in list(ind.ravel()) ])
    

single fertilization
developmental process
cellular anatomical entity
multi-organism reproductive process
plasma membrane fusion
molecular_function
intrinsic component of plasma membrane
cellular process involved in reproduction in multicellular organism
membrane
membrane-bounded organelle
cellular developmental process
reproduction
organelle
intrinsic component of membrane
endoplasmic reticulum membrane
integral component of plasma membrane
binding
plasma membrane
sexual reproduction
fertilization
biological_process
pollen sperm cell differentiation
reproductive process
double fertilization forming a zygote and endosperm
integral component of membrane
intracellular organelle
cellular component organization
membrane fusion
lipid binding
multi-organism process
intracellular membrane-bounded organelle
cell differentiation
membrane organization
fusion of sperm to egg plasma membrane involved in double fertilization forming a zygote and endosperm
cellular process
developmental process invo

In [74]:
#create a normalized taxonomy embedding for a species aware classifier
print( len ( [ l for l in p.tree.get_leaves() ]))
import time
leaves = [ l for l in p.tree.get_leaves() ]
import random

times =[]
for i in range(100):
    t = time.time()
    p.tree.get_distance(random.choice(leaves).name,random.choice(leaves).name, topology_only=True)
    times.append(time.time()-t)
plt.hist(times)
print( np.mean(times)*(2290**2 / 2 - 2290) / 60 )

2290


NameError: name 'plt' is not defined

In [75]:

import multiprocessing as mp
from functools import partial

pool = mp.Pool()
distances = [ (l1.name,l2.name ) for i,l1 in enumerate(p.tree.get_leaves()) for j,l2 in enumerate(p.tree.get_leaves()) if i < j  ]
indices = [ (i,j )   for i,l1 in enumerate(p.tree.get_leaves()) for j,l2 in enumerate(p.tree.get_leaves()) if i < j  ]

def treedist( ltuple , tree):
    return tree.get_distance( ltuple[0] , ltuple[1], topology_only = True)

nleaves = len([ l for l in p.tree.get_leaves() ] )
treedist_load = partial( treedist , tree= p.tree )
res = pool.map_async(   treedist_load , distances ).get()


OSError: [Errno 12] Cannot allocate memory

In [77]:
taxmat = np.zeros( (nleaves, nleaves) )
for i,index in enumerate(indices):
    taxmat[index] = res[i]
taxmat+=taxmat.T

print(taxmat)
plt.hist(taxmat.ravel())
plt.show()
with open( 'taxkernel.pkl' ,'wb')as taxkernelout:
    taxkernelout.write(pickle.dumps(distances))
#project down with mds

NameError: name 'nleaves' is not defined

In [ ]:
from sklearn.manifold import MDS
#create taxonomy projection
mds = MDS( n_components = 2 )
taxembed = mds.fit_transform(taxmat)
with open( 'taxproj.pkl' , 'wb') as taxprojout:
    taxprojout.write( pickle.dumps(taxembed))

In [ ]:
with open( 'taxproj.pkl' , 'rb' ) as taxprojin :
    taxembed = pickle.loads( taxprojin.read())
    
#use the profiler trimmed ncbi tree to generate an index 
taxindex = {int( n.name ) : i for i,n in enumerate( p.tree.get_leaves() ) }

plt.scatter( taxembed[:,0], taxembed[:,1]  )
#where is arabidopsis
plt.scatter( taxembed[taxindex[3702],0] , taxembed[taxindex[3702],1])
#where is human
plt.scatter( taxembed[taxindex[9606],0] , taxembed[taxindex[9606],1])

plt.show()

In [ ]:
from sklearn.preprocessing import RobustScaler , MinMaxScaler

rs = MinMaxScaler()
taxembedscaled = rs.fit_transform( taxembed )

plt.scatter( taxembedscaled[:,0], taxembedscaled[:,1]  )
plt.show()

In [ ]:
embedmat_scaled = rs.fit_transform( embedmat )


In [90]:
from sklearn.cluster import AffinityPropagation

def fam2hogid(fam_id):
    """
    For use with OMA HOGs
    Get hog id given fam
    :param fam_id: fam
    :return: hog id
    """
    hog_id = "HOG:" + (7-len(str(fam_id))) * '0' + str(fam_id)
    return hog_id


def profiler_naive( qhog , p , gaf , c , nannot ,  k = 10 , verbose = True , ):
    #use jaccard and IC to weigh all returned GO terms    
    
    res = p.hog_query(fam_id = qhog , k = k )
        
    #grab entries from hogs
    hashes =  p.pull_hashes(res+[qhog])
    jaccards = { r:hashes[r].jaccard(hashes[qhog]) for r in res }
    
    members =  { r : [m for m in p.db_obj.iter_members_of_hog_id(fam2hogid(r) , ) if m.omaid in gaf ] for r in res  }
    godict = {  m.omaid + '_' + str(go) :{ 'fam':r , 'GO':go , 'Tax':m.genome , 'Jaccard': jaccards[r] } for r in res for m in members[r] for go in get_ancestors( gaf[m.omaid] , obo )  }
    godf = pd.DataFrame.from_dict( godict , orient = 'index')
    
    ICs = { go:np.log( nannot/c[go]) for go in godf.GO.unique() if go in c }
    godf['IC'] = godf.GO.map(ICs)
    #calculate the jaccards relative to the query
    
    godf['jaccard'] = godf.fam.map(jaccards)     
    #max jaccard * IC
    
    preddict = { go:{'max_jaccard':godf[godf.GO == go].jaccard.max() , 'IC':ICs[go]  } for go in godf.GO.unique() if go in ICs }
    predDF = pd.DataFrame.from_dict( preddict , orient = 'index')
    
    
    #awesome master super scoring function goes here
    predDF['score'] =  predDF.max_jaccard * predDF.IC
    predDF['name'] = predDF.index.map(lambda go : obo[ id2GO(go)].name )
    
    #consequences be damned...
    return predDF


def embedding_kd_dbscan( p, kdtree, embedmat ):
    
    res = p.hog_query(fam_id = qhog , k = k )
    #grab entries from hogs leaving qhog in there
    godict = {  m.omaid + '_' + str(go) :{ 'fam':r , 'GO':go , 'Tax':m.genome , 'Jaccard': jaccards[r] } for r in res for m in members[r] for go in get_ancestors( gaf[m.omaid] , obo )  }
    
    
    godf = pd.DataFrame.from_dict( godict , orient = 'index')
    
    GO_embedding = { go:embedmat[ index[go], : ] for go in godf.unique() }
    
    godf['Goembed'] = godf.GO.map(GO_embedding)
    
    ICs = { go:np.log( nannot/c[go]) for go in godf.GO.unique() }

    #calculate the jaccards relative to the query
    hashes =  p.pull_hashes(res+[qhog])
    jaccards = { r:hashes[r].jaccard(hashes[qhog]) for r in res }
    
    #AP into clusters
    af = AffinityPropagation(preference=-50).fit(np.vstack(godf.GOembed))
    cluster_centers_indices = af.cluster_centers_indices_
    
    #take cluster centroids 
    for clustercenter in cluster_centers_indices:
        #use embedding kdtree to find all close terms to cluster centroids
        suggestions =  dict( zip(['dist','index'] , tree.query( GO_embedding[go] , k=10) ))
        
    
    #awesome super scoring function goes here. maybe give the suggestions a bit less weight
    preDF['score'] =  0
        #consequences be damned...
    #output using naive jaccard style confidence with IC and jaccard score

    return predDF



In [91]:
#generate predictions with profiler naive
fams = list(targets.fam)
print(len(fams))

nannot = sum(c.values())
print(nannot)
for h in fams[0:10]:
    print('query:' +str(h) )
    print(profiler_naive( h , p , gaf , c , nannot , k= 10  ))
#generate predictions with profiler and go embedding



83525
11603659850
query:472976
         max_jaccard         IC      score  \
122         0.539062  12.990706   7.002802   
138         0.531250  17.832252   9.473384   
139         1.000000  11.509970  11.509970   
166         0.531250   5.708596   3.032692   
281         0.511719  13.903621   7.114744   
313         0.515625  13.332655   6.874650   
315         0.515625  14.185017   7.314150   
910         0.511719  10.934818   5.595551   
976         0.539062  12.049370   6.495363   
977         0.539062  12.409497   6.689494   
981         0.539062  10.728985   5.783594   
1012        0.539062  12.402634   6.685795   
1067        0.539062  11.794884   6.358180   
1217        0.539062  14.553213   7.845092   
1227        0.539062  14.655395   7.900174   
1533        0.511719  16.111538   8.244576   
1701        0.531250  15.682383   8.331266   
2376        0.496094   9.549589   4.737491   
3333        0.523438  10.258001   5.369422   
3674        1.000000   3.700473   3.700473   
367

KeyError: 'GO:1990967'

In [60]:
###### start machine learning profiler based function prediction here #####
#fit distribution params to jaccards and annot vecs?
#pull in terms in the embedding and use them as suggestions

def gengovec( qseqdfs , gaf , c , taxembed , embedmat , index  , visdist = False , k = 10 ):
    nannot = sum(c.values())
    #prob a good idea to generate a bunch of these and fit a normalizer on the data.
    #qseqdata is a generator of dataframes with columns = [ qseq , qhog ]
    #sequences are ordered by Hog number to avoid splitting them up
    for qseqdata in qseqdfs:
        for qhog in qseqdata.hogs.uniqe():
            #proceed by hog
            #use the profiler to get all coev HOGs
            res = p.hog_query(fam_id = qhog , k = k )
            if qhog in res:
                res.remove( qhog )

            #calculate the jaccards relative to the query
            hashes =  p.pull_hashes(res+[qhog])
            jaccards = { r:hashes[r].jaccard(hashes[qhog]) for r in res }

            Jaccardavg = np.mean( jaccards.values()  )
            Jaccardstdv = np.stdv( jaccards.values() )

            #if no annotations then just use 'biological process' as annotation
            #calculate Hog annotation centroid
            qHOGgo =  [ embedmat[ index[go] ,:]  for m in p.db_obj.iter_members_of_hog_id(qhog) for go in gaf[m.omaid] if m.omaid in gaf ]

            if len(qHOGgo) == 0 :
                #if no annotations then just use 'biological process' as annotation
                qHOGavg = embedmat[ index[7582] ,: ]
                qHOGstdv = [0,0,0]
            else:
                #use embbedmat here
                
                qHOGavg = np.mean( np.vstack(qHOGgo))
                qHOGstdv = np.stdv( np.vstack(qHOGgo))

            members =  { r : [m for m in p.db_obj.iter_members_of_hog_id(r) if m.omaid in gaf ] for r in res  }
            #each annot to each prot
            godict = {  m.omaid + '_' + str(go) :{ 'fam':r , 'GO':go , 'Tax':m.genome , 'Jaccard': jaccards[r] } for r in res for m in members[r] for go in get_ancestors( gaf[m.omaid] , obo )  }
            godf = pd.DataFrame.from_dict( godict , orient = 'index')
            GO_embedding = { go:embedmat[ index[go], : ] for go in godf.unique() }

            godf['Goembed'] = godf.GO.map(GO_embedding)

            Counts = { go:len(godf[godf.GO==go]) for go in godf.GO.unique() }

            Freqs = { go:Counts[go]/len(godf) for go in godf.GO.unique() }

            ICs = { go:np.log( nannot/c[go]) for go in godf.GO.unique() }

            godf['IC'] = godf.GO.map(ICs)

            Tax_embedding = { genome:taxkernel[taxindex[genome],:] for genome in godf.Tax.unique() }

            godf['TaxEmbed'] = godf.Tax.map( Tax_embedding )

            GOembedavg = np.mean( np.vstack(godf.GOembed) ) 

            GOembedstd = np.mean( np.vstack(godf.GOembed) )

            ICavg = np.mean( godf.IC )

            ICstdv = np.stdv( godf.IC )

            Countavg = np.mean( counts.values() )
            Countstdv = np.stdv( counts.values() )

            if vis_dist:
                plt.title('Jaccard')
                plt.hist(sub.jaccard)
                plt.show()

                plt.title('IC')
                plt.hist(ICs.values())
                plt.show()

            for seq in qseqdata[qseqdata.hogs == qhog ].seqs.unique():
                #for each go for each qseq
                for go in godf.GO.unique():

                    sub = godf[ godf.GO == go ]

                    seqTaxEmbed = taxmat[ taxindex.loc[qseq].genome , :  ]
                    #min taxonomic distance at which the annotation being tested is found
                    if taxindex.loc[qseq].genome in sub.genome:
                        taxdist = 0
                    else:
                        taxdist = np.amin( np.norm( sub.Taxembed - np.tile( seqTaxEmbed (1,len(sub))) ) )

                    #generate a vec
                    #infomation content of proposed term
                    #todo: max IC, min IC

                    #avg IC of returned results
                    #stdv IC of returned results

                    #todo: max count, min count

                    #avg Count of annotation in results
                    #stdv Count of annotations in results
                    #Freq of term proposed

                    #Avg embedding of returned results
                    #stdv embedding of returned results

                    #Avg embedding of qhog annotations
                    #stdv embedding of qhog annotations

                    #max jaccard of HOG w annotation against qhog
                    #max jaccard across returned results
                    #mean jaccard across returned results
                    #stdv of jaccard across returned results


                    govec = np.arry([ ICs[go] ,  ICavg , ICstdv , Countavg, Countstdv , Counts[go] , Freqs[go], GOembedavg, Goembedstdv , GO_embedding[go] 
                                     , qHOGavg, qHOGstdv, sub.Jaccard.max() , jaccardsmean, jaccardstdv, godf.Jaccard.max(), godf.Jaccard.min(), 
                                     taxdist , seqTaxEmbed , suggestiondist  ] )

                    if go in qHOGgo :
                        y = [1]
                    else:
                        y =[0]

                    yield govec , y




In [ ]:
#generate a bunch of samples and fit robust scaler 
from sklearn.preprocessing import RobustScaler
RS = RobustScaler()


#pick random hogs with annotations


#fit scaler



In [ ]:
#use this net for predicting binary yes or no for each go term in a set given a set of inputs
import numpy as np
from keras.optimizers import *
from keras.layers import *
from keras.callbacks import *
from keras.models import load_model
import datetime
import keras
global max_epochs
from keras.models import Model
from keras.layers import Flatten, Input, Dense

from keras.backend import tensorflow_backend as K
epsilon = K.epsilon()

import tensorflow as tf


################################################################################
dirlocal = './'
dataset = 'full' # 'sample' or 'full'
 
modelfile = 'model-' + str(stamp) + '.h5'

densedim = 30
layers = 10 

valentries = [ random.choice(gaf.keys()) for i in range( 1000 ) ]
valqseqdata = { }
valmat = [ gengovec( qseqdata , gaf , c , taxembed , embedmat , go_index , tax_index , visdist = False , k = 10 ) for qseqdata in valqseqdata ]

xval, yval = zip(*valmat)
xval= np.vstack(xval)
yval = np.vastack(yval)


a = Input(shape=(len(govec_test,))
for i in range( layers):
    if i == 0:
        b = Dense(densedim)(a)
        layer = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones')
        b = layer(b)

    else:
        #add batch reg
        #add dropout
        b = Dense(densedim)(b)
        layer = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones')
        b = layer(b)
final = Dense(1,activation =  'softmax')
          

model = Model(inputs=a, outputs=final)

o = keras.optimizers.Adadelta(lr=.025, rho=0.95)
model.compile( optimizer=o, loss= 'binary cross entropy' , metrics=['mae'])

max_len = 3000
mc = ModelCheckpoint(modelfile, monitor = 'val_loss', mode = 'min', verbose = 1, save_best_only = True)
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 2, patience = es_patience)
          
####change this
history = model.fit_generator( { 'Seqin': XTRAINrows } , [YTRAINpca,YTRAIN] , batch_size = None , verbose = 2 , epochs = 100000 ,  validation_data=([XVALIDrows], [YVALIDpca,YVALID] ), callbacks=[es, mc])



In [ ]:

# jaccard , IC , depth , vec embed , avg of GO terms embed , variance of go terms embed,  num go terms returned ,  profile DB type (one hot for each DB made)

#use batch normalization to scale input values

# possible input: difference and intersection profile vector with all hogs that have the term
#would have to iron out the profile storage issue before this works


inputlayer = Input( )



